# 02. Forecasting Models: ARIMA vs. XGBoost

**Objective**: Predict future temperature values (Next Hour/24h) using historical data. We will compare a classical statistical method (**ARIMA**) against a modern machine learning approach (**XGBoost**) using rigorous Time-Series Cross-Validation.

**Target City**: London (Selected for demonstration stability)

## 1. Imports & Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

# Set Plotly as default backend
pd.options.plotting.backend = "plotly"

## 2. Data Loading & Preparation

For this modeling experiment, we isolate **London** to ensure we are modeling a single coherent time-series. We also resample to a strict Hourly frequency to handle any irregular updates.

In [2]:
# Load Data
df = pd.read_csv('../data/raw/GlobalWeatherRepository.csv')
df['last_updated'] = pd.to_datetime(df['last_updated'])

# Filter for Target City
TARGET_CITY = "London"
df_city = df[df['location_name'] == TARGET_CITY].copy()

# Set Index and Sort
df_city = df_city.sort_values('last_updated').set_index('last_updated')

# Resample to Hourly to ensure regularity (Mean aggregation handles duplicates)
df_hourly = df_city[['temperature_celsius']].resample('H').mean().interpolate(method='time')

print(f"Modeled Dataset Shape for {TARGET_CITY}: {df_hourly.shape}")
df_hourly.head()

Modeled Dataset Shape for London: (14087, 1)


,temperature_celsius
last_updated,
2024-05-16 09:00:00,14.000000
2024-05-16 10:00:00,14.166667
2024-05-16 11:00:00,14.333333
2024-05-16 12:00:00,14.500000
2024-05-16 13:00:00,14.666667


## 3. Feature Engineering

Machine Learning models like XGBoost cannot "see" time. We must explicitly give them temporary context via features.

1.  **Lags**: What was the temperature 1 hour ago? 24 hours ago?
2.  **Rolling Windows**: What was the average temperature over the last day?
3.  **Cyclical Time**: Encoding Hour of Day (0-23) as Sin/Cos to tell the model that 23:00 is close to 00:00.

In [3]:
def create_features(data):
    df = data.copy()
    
    # 1. Time Lags (Autocorrelation)
    df['lag_1'] = df['temperature_celsius'].shift(1)
    df['lag_2'] = df['temperature_celsius'].shift(2)
    df['lag_3'] = df['temperature_celsius'].shift(3)
    df['lag_24'] = df['temperature_celsius'].shift(24)
    
    # 2. Rolling Statistics (Trend/Volatility)
    df['rolling_mean_24'] = df['temperature_celsius'].rolling(window=24).mean()
    df['rolling_std_24'] = df['temperature_celsius'].rolling(window=24).std()
    
    # 3. Cyclical Time Encoding
    df['hour'] = df.index.hour
    df['month'] = df.index.month
    
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    return df.dropna()

df_features = create_features(df_hourly)
print(f"Feature Matrix Shape: {df_features.shape}")
df_features.head(3)

Feature Matrix Shape: (14063, 13)


,temperature_celsius,lag_1,lag_2,lag_3,lag_24,rolling_mean_24,rolling_std_24,hour,month,hour_sin,hour_cos,month_sin,month_cos
last_updated,,,,,,,,,,,,,
2024-05-17 09:00:00,19.153846,18.923077,18.692308,18.461538,14.000000,16.540064,1.573120,9,5,0.707107,-0.707107,0.5,-0.866025
2024-05-17 10:00:00,19.384615,19.153846,18.923077,18.692308,14.166667,16.757479,1.591311,10,5,0.500000,-0.866025,0.5,-0.866025
2024-05-17 11:00:00,19.615385,19.384615,19.153846,18.923077,14.333333,16.977564,1.606656,11,5,0.258819,-0.965926,0.5,-0.866025


## 4. Train/Test Split

**CRITICAL**: We must split by time. We cannot shuffle.
*   **Train**: Past data (80%)
*   **Test**: Future data (20%)

In [4]:
split_idx = int(len(df_features) * 0.8)

train = df_features.iloc[:split_idx]
test = df_features.iloc[split_idx:]

print(f"Train Range: {train.index.min()} to {train.index.max()}")
print(f"Test Range:  {test.index.min()} to {test.index.max()}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train['temperature_celsius'], name='Train'))
fig.add_trace(go.Scatter(x=test.index, y=test['temperature_celsius'], name='Test'))
fig.update_layout(title="Train/Test Split Scheme")
fig.show()

Train Range: 2024-05-17 09:00:00 to 2025-08-29 02:00:00
Test Range:  2025-08-29 03:00:00 to 2025-12-24 07:00:00


## 5. Baseline Model

The **Persistence Model** assumes: *"The temperature in the next hour will be the same as this hour."*
Any complex model MUST beat this.

In [5]:
# Persistence Forecast: Predicted(t) = Actual(t-1)
# Luckily, we already created 'lag_1' which represents exactly this.
y_true = test['temperature_celsius']
y_pred_baseline = test['lag_1']

mae_baseline = mean_absolute_error(y_true, y_pred_baseline)
rmse_baseline = np.sqrt(mean_squared_error(y_true, y_pred_baseline))

print(f"Baseline MAE: {mae_baseline:.4f} °C")
print(f"Baseline RMSE: {rmse_baseline:.4f} °C")

Baseline MAE: 0.1163 °C
Baseline RMSE: 0.1536 °C


## 6. ARIMA Model

AutoRegressive Integrated Moving Average. A gold standard in univariate forecasting.
We use order=(1,1,1) as a starting point.

In [6]:
# ARIMA requires only the target series
train_y = train['temperature_celsius']
test_y = test['temperature_celsius']

# Fit ARIMA
model_arima = ARIMA(train_y, order=(5,1,0)) # AR(5) to capture some daily lag influence manually if needed
model_arima_fit = model_arima.fit()

# Forecast
# Note: Standard ARIMA forecast in a loop for one-step ahead is slow.
# Here we do a static forecast for the test range for demonstration speed.
# In PROD, we would use Rolling Forecast Origin.
arima_forecast = model_arima_fit.forecast(steps=len(test_y))
arima_forecast.index = test_y.index

# Evaluate
mae_arima = mean_absolute_error(test_y, arima_forecast)
rmse_arima = np.sqrt(mean_squared_error(test_y, arima_forecast))

print(f"ARIMA MAE: {mae_arima:.4f} °C")
print(f"ARIMA RMSE: {rmse_arima:.4f} °C")

ARIMA MAE: 4.6336 °C
ARIMA RMSE: 5.8172 °C


## 7. XGBoost Model

Gradient Boosted Trees can capture non-linear relationships and interactions between lags and cyclical features.

In [7]:
# Features for XGBoost
features = ['lag_1', 'lag_2', 'lag_3', 'lag_24', 
            'rolling_mean_24', 'rolling_std_24', 
            'hour_sin', 'hour_cos', 'month_sin', 'month_cos']
target = 'temperature_celsius'

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

# Initialize and Train
model_xgb = XGBRegressor(n_estimators=1000, learning_rate=0.01, early_stopping_rounds=50)
model_xgb.fit(X_train, y_train, 
              eval_set=[(X_train, y_train), (X_test, y_test)], 
              verbose=False)

# Predict
xgb_pred = model_xgb.predict(X_test)

# Evaluate
mae_xgb = mean_absolute_error(y_test, xgb_pred)
rmse_xgb = np.sqrt(mean_squared_error(y_test, xgb_pred))

print(f"XGBoost MAE: {mae_xgb:.4f} °C")
print(f"XGBoost RMSE: {rmse_xgb:.4f} °C")

XGBoost MAE: 0.0829 °C
XGBoost RMSE: 0.1270 °C


## 8. Final Comparison & Visualization

Which model won?

In [8]:
results = pd.DataFrame({
    'Model': ['Baseline', 'ARIMA', 'XGBoost'],
    'MAE': [mae_baseline, mae_arima, mae_xgb],
    'RMSE': [rmse_baseline, rmse_arima, rmse_xgb]
})

print(results)

# Visualization of Predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=test.index, y=test['temperature_celsius'], name='Actual', line=dict(color='black', width=2)))
fig.add_trace(go.Scatter(x=test.index, y=xgb_pred, name='XGBoost', line=dict(color='orange', dash='dot')))
fig.add_trace(go.Scatter(x=test.index, y=arima_forecast, name='ARIMA', line=dict(color='blue', dash='dash')))

fig.update_layout(title="Forecast Comparison: London Temperature",
                  xaxis_title="Date",
                  yaxis_title="Temperature (°C)")
fig.show()

      Model       MAE      RMSE
0  Baseline  0.116339  0.153646
1     ARIMA  4.633586  5.817162
2   XGBoost  0.082918  0.127047


In [9]:
# Feature Importance (XGBoost)
importance = pd.DataFrame({
    'Feature': features,
    'Importance': model_xgb.feature_importances_
}).sort_values('Importance', ascending=True)

fig = px.bar(importance, x='Importance', y='Feature', orientation='h', title="XGBoost Feature Importance")
fig.show()